# ISIC 2018 Klasifikasyon Pipeline (Colab Versiyonu)

Bu notebook, Google Colab üzerinde uçtan uca çalışacak şekilde tasarlanmıştır.
Otomatik olarak:
1. Kütüphaneleri kurar.
2. Kaggle API ile veriyi indirir.
3. Gerekli modülleri oluşturur.
4. Eğitimi ve değerlendirmeyi başlatır.

## 1. Hazırlık ve Kurulum

In [1]:
# Gerekli kütüphaneleri yükle
!pip install -q tensorflow pandas matplotlib seaborn scikit-learn kagglehub tf-keras-vis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 993.2 kB/s eta 0:00:00


In [2]:
# Veri Setini İndir (kagglehub ile)
import kagglehub

# Download latest version
path = kagglehub.dataset_download("nodoubttome/skin-cancer9-classesisic")

print("Path to dataset files:", path)

# Path'i global değişkende sakla
DATASET_PATH = path

Using Colab cache for faster access to the 'skin-cancer9-classesisic' dataset.
Path to dataset files: /kaggle/input/skin-cancer9-classesisic


In [3]:
# Kaynak kod klasörünü oluştur
!mkdir -p src

In [4]:
%%writefile src/__init__.py
"""
ISIC 2018 Skin Lesion Classification Package
"""
from . import config
from . import data_loader
from . import models
from . import training
from . import evaluation
from . import gradcam

__version__ = "0.1.0"
__all__ = ["config", "data_loader", "models", "training", "evaluation", "gradcam"]


Writing src/__init__.py


In [5]:
%%writefile src/config.py
"""
ISIC 2018 Skin Lesion Classification - Configuration Module
"""
import os
import random
import numpy as np

# Random seed for reproducibility
SEED = 42

# Paths
BASE_DIR = '/content'
DATA_DIR = os.path.join(BASE_DIR, 'isic_data')
TRAIN_DIR = os.path.join(DATA_DIR, "Train")
TEST_DIR = os.path.join(DATA_DIR, "Test")
OUTPUT_DIR = os.path.join(BASE_DIR, "outputs")
MODELS_DIR = os.path.join(OUTPUT_DIR, "models")
FIGURES_DIR = os.path.join(OUTPUT_DIR, "figures")
GRADCAM_DIR = os.path.join(OUTPUT_DIR, "gradcam")
REPORTS_DIR = os.path.join(OUTPUT_DIR, "reports")

# Image parameters
IMG_SIZE = (224, 224)
IMG_SHAPE = (224, 224, 3)

# Data split ratios
TRAIN_RATIO = 0.70
VAL_RATIO = 0.15
TEST_RATIO = 0.15

# Training parameters
BATCH_SIZE = 32
EPOCHS = 100
EARLY_STOPPING_PATIENCE = 10
REDUCE_LR_PATIENCE = 5
REDUCE_LR_FACTOR = 0.2
MIN_LR = 1e-6

# Learning rates
SCRATCH_LR = 1e-3
TL_FREEZE_LR = 1e-3
TL_FINETUNE_LR = 1e-5

# Fine-tuning: unfreeze last 25% of layers
FINETUNE_RATIO = 0.25

# Augmentation parameters
ROTATION_RANGE = 15  # degrees
ZOOM_RANGE = 0.1
BRIGHTNESS_RANGE = 0.1
HORIZONTAL_FLIP = True
VERTICAL_FLIP = True

# Class mapping (will be populated after dataset analysis)
CLASS_NAMES = []
BINARY_LABELS = {}


def set_seed(seed: int = SEED):
    """Set random seed for reproducibility."""
    random.seed(seed)
    np.random.seed(seed)
    try:
        import tensorflow as tf
        tf.random.set_seed(seed)
        # Set deterministic operations (may impact performance)
        os.environ['TF_DETERMINISTIC_OPS'] = '1'
    except ImportError:
        pass


def ensure_dirs():
    """Create output directories if they don't exist."""
    for dir_path in [MODELS_DIR, FIGURES_DIR, GRADCAM_DIR, REPORTS_DIR]:
        os.makedirs(dir_path, exist_ok=True)


# Initialize
set_seed()
ensure_dirs()


Writing src/config.py


In [6]:
%%writefile src/data_loader.py
"""
ISIC 2018 Skin Lesion Classification - Data Loading Module
"""
import os
from collections import Counter
from pathlib import Path
from typing import Tuple, List, Dict

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split

from . import config


def analyze_dataset() -> pd.DataFrame:
    """
    Analyze the dataset and return class distribution.

    Returns:
        DataFrame with class names and sample counts
    """
    class_counts = {}

    for class_name in os.listdir(config.TRAIN_DIR):
        class_path = os.path.join(config.TRAIN_DIR, class_name)
        if os.path.isdir(class_path):
            count = len([f for f in os.listdir(class_path)
                        if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
            class_counts[class_name] = count

    df = pd.DataFrame([
        {'class_name': k, 'count': v}
        for k, v in class_counts.items()
    ]).sort_values('count', ascending=False).reset_index(drop=True)

    print("\n" + "="*60)
    print("ISIC 2018 Dataset - Class Distribution")
    print("="*60)
    print(df.to_string(index=False))
    print(f"\nTotal samples: {df['count'].sum()}")
    print("="*60)

    return df


def get_top_two_classes() -> Tuple[str, str]:
    """
    Get the two classes with the highest sample counts.

    Returns:
        Tuple of (class_0_name, class_1_name)
    """
    df = analyze_dataset()
    top_two = df.head(2)['class_name'].tolist()

    print(f"\nSelected classes for binary classification:")
    print(f"  Class 0: {top_two[0]}")
    print(f"  Class 1: {top_two[1]}")

    return top_two[0], top_two[1]


def load_image_paths_and_labels(class_0: str, class_1: str) -> Tuple[List[str], List[int]]:
    """
    Load image paths and binary labels for the selected two classes.

    Args:
        class_0: Name of class to be labeled as 0
        class_1: Name of class to be labeled as 1

    Returns:
        Tuple of (image_paths, labels)
    """
    image_paths = []
    labels = []

    # Load class 0
    class_0_path = os.path.join(config.TRAIN_DIR, class_0)
    for img_name in os.listdir(class_0_path):
        if img_name.lower().endswith(('.jpg', '.jpeg', '.png')):
            image_paths.append(os.path.join(class_0_path, img_name))
            labels.append(0)

    # Load class 1
    class_1_path = os.path.join(config.TRAIN_DIR, class_1)
    for img_name in os.listdir(class_1_path):
        if img_name.lower().endswith(('.jpg', '.jpeg', '.png')):
            image_paths.append(os.path.join(class_1_path, img_name))
            labels.append(1)

    print(f"\nLoaded {len(image_paths)} images:")
    print(f"  Class 0 ({class_0}): {labels.count(0)}")
    print(f"  Class 1 ({class_1}): {labels.count(1)}")

    return image_paths, labels


def create_stratified_split(
    image_paths: List[str],
    labels: List[int]
) -> Dict[str, Tuple[List[str], List[int]]]:
    """
    Create stratified train/validation/test splits.

    Args:
        image_paths: List of image file paths
        labels: List of corresponding labels

    Returns:
        Dictionary with 'train', 'val', 'test' keys containing (paths, labels) tuples
    """
    # First split: train+val vs test
    train_val_paths, test_paths, train_val_labels, test_labels = train_test_split(
        image_paths, labels,
        test_size=config.TEST_RATIO,
        stratify=labels,
        random_state=config.SEED
    )

    # Second split: train vs val
    val_ratio_adjusted = config.VAL_RATIO / (config.TRAIN_RATIO + config.VAL_RATIO)
    train_paths, val_paths, train_labels, val_labels = train_test_split(
        train_val_paths, train_val_labels,
        test_size=val_ratio_adjusted,
        stratify=train_val_labels,
        random_state=config.SEED
    )

    splits = {
        'train': (train_paths, train_labels),
        'val': (val_paths, val_labels),
        'test': (test_paths, test_labels)
    }

    print("\n" + "="*60)
    print("Stratified Split Results")
    print("="*60)
    for split_name, (paths, lbls) in splits.items():
        class_dist = Counter(lbls)
        print(f"{split_name.upper():>10}: {len(paths):>4} samples | "
              f"Class 0: {class_dist[0]:>3} | Class 1: {class_dist[1]:>3}")
    print("="*60)

    return splits


def create_augmentation_layer() -> tf.keras.Sequential:
    """
    Create data augmentation layer for training.

    Returns:
        Sequential model containing augmentation layers
    """
    return tf.keras.Sequential([
        tf.keras.layers.RandomFlip("horizontal_and_vertical"),
        tf.keras.layers.RandomRotation(config.ROTATION_RANGE / 360.0),
        tf.keras.layers.RandomZoom(config.ZOOM_RANGE),
        tf.keras.layers.RandomBrightness(config.BRIGHTNESS_RANGE),
        tf.keras.layers.RandomContrast(config.BRIGHTNESS_RANGE),
    ], name='augmentation')


def preprocess_image(path: str, label: int) -> Tuple[tf.Tensor, int]:
    """
    Load and preprocess a single image.

    Args:
        path: Image file path
        label: Image label

    Returns:
        Tuple of (preprocessed_image, label)
    """
    # Read image
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    # Resize
    img = tf.image.resize(img, config.IMG_SIZE)
    # Normalize to [0, 1]
    img = tf.cast(img, tf.float32) / 255.0

    return img, label


def create_dataset(
    paths: List[str],
    labels: List[int],
    is_training: bool = False,
    batch_size: int = None
) -> tf.data.Dataset:
    """
    Create a tf.data.Dataset pipeline.

    Args:
        paths: List of image paths
        labels: List of labels
        is_training: Whether to apply augmentation and shuffle
        batch_size: Batch size (uses config default if None)

    Returns:
        tf.data.Dataset
    """
    if batch_size is None:
        batch_size = config.BATCH_SIZE

    # Create dataset from tensors
    dataset = tf.data.Dataset.from_tensor_slices((paths, labels))

    # Shuffle if training
    if is_training:
        dataset = dataset.shuffle(buffer_size=len(paths), seed=config.SEED)

    # Map preprocessing (parallel)
    dataset = dataset.map(
        preprocess_image,
        num_parallel_calls=tf.data.AUTOTUNE
    )

    # Batch
    dataset = dataset.batch(batch_size)

    # Apply augmentation if training (after batching)
    if is_training:
        augmentation = create_augmentation_layer()
        dataset = dataset.map(
            lambda x, y: (augmentation(x, training=True), y),
            num_parallel_calls=tf.data.AUTOTUNE
        )

    # Cache and prefetch
    dataset = dataset.cache().prefetch(tf.data.AUTOTUNE)

    return dataset


def visualize_augmentation(
    dataset: tf.data.Dataset,
    class_names: Tuple[str, str],
    num_samples: int = 6,
    save_path: str = None
):
    """
    Visualize augmented samples.

    Args:
        dataset: Training dataset with augmentation
        class_names: Tuple of class names
        num_samples: Number of samples to visualize
        save_path: Path to save the figure
    """
    # Get one batch
    for images, labels in dataset.take(1):
        fig, axes = plt.subplots(2, 3, figsize=(12, 8))
        fig.suptitle('Data Augmentation Examples', fontsize=14)

        for i, ax in enumerate(axes.flat):
            if i < len(images):
                ax.imshow(images[i].numpy())
                ax.set_title(f'Class: {class_names[labels[i].numpy()]}')
                ax.axis('off')

        plt.tight_layout()

        if save_path:
            plt.savefig(save_path, dpi=150, bbox_inches='tight')
            print(f"Augmentation examples saved to: {save_path}")

        plt.close()


def prepare_data() -> Tuple[tf.data.Dataset, tf.data.Dataset, tf.data.Dataset, Tuple[str, str], Dict]:
    """
    Main function to prepare all data.

    Returns:
        Tuple of (train_ds, val_ds, test_ds, class_names, split_info)
    """
    # Get top two classes
    class_0, class_1 = get_top_two_classes()
    class_names = (class_0, class_1)

    # Load paths and labels
    image_paths, labels = load_image_paths_and_labels(class_0, class_1)

    # Create stratified splits
    splits = create_stratified_split(image_paths, labels)

    # Create datasets
    train_ds = create_dataset(
        splits['train'][0], splits['train'][1],
        is_training=True
    )
    val_ds = create_dataset(
        splits['val'][0], splits['val'][1],
        is_training=False
    )
    test_ds = create_dataset(
        splits['test'][0], splits['test'][1],
        is_training=False
    )

    # Save augmentation examples
    visualize_augmentation(
        train_ds, class_names,
        save_path=os.path.join(config.FIGURES_DIR, 'augmentation_examples.png')
    )

    # Prepare split info for reporting
    split_info = {
        'train': {'total': len(splits['train'][0]), 'class_0': splits['train'][1].count(0), 'class_1': splits['train'][1].count(1)},
        'val': {'total': len(splits['val'][0]), 'class_0': splits['val'][1].count(0), 'class_1': splits['val'][1].count(1)},
        'test': {'total': len(splits['test'][0]), 'class_0': splits['test'][1].count(0), 'class_1': splits['test'][1].count(1)},
        'class_names': class_names
    }

    # Store test paths for later Grad-CAM analysis
    split_info['test_paths'] = splits['test'][0]
    split_info['test_labels'] = splits['test'][1]

    return train_ds, val_ds, test_ds, class_names, split_info


if __name__ == "__main__":
    # Test the data loading
    train_ds, val_ds, test_ds, class_names, split_info = prepare_data()

    print("\nDataset shapes:")
    for x, y in train_ds.take(1):
        print(f"  Batch shape: {x.shape}")
        print(f"  Labels shape: {y.shape}")


Writing src/data_loader.py


In [7]:
%%writefile src/models.py
"""
ISIC 2018 Skin Lesion Classification - Models Module
"""
import tensorflow as tf
from tensorflow import keras
from keras import layers, Model

from . import config


def create_scratch_cnn(input_shape: tuple = None) -> Model:
    """
    Create a Scratch CNN model as specified in the project requirements.

    Architecture:
    - Block 1: Conv(32) x2 + MaxPool + Dropout
    - Block 2: Conv(64) x2 + MaxPool + Dropout
    - Block 3: Conv(128) x2 + MaxPool + Dropout
    - Block 4: Conv(256) + MaxPool + Dropout
    - Head: GAP + Dense(128) + Dense(1)

    Args:
        input_shape: Input image shape (default: from config)

    Returns:
        Compiled Keras model
    """
    if input_shape is None:
        input_shape = config.IMG_SHAPE

    inputs = layers.Input(shape=input_shape, name='input')

    # Block 1
    x = layers.Conv2D(32, (3, 3), padding='same', name='block1_conv1')(inputs)
    x = layers.BatchNormalization(name='block1_bn1')(x)
    x = layers.ReLU(name='block1_relu1')(x)
    x = layers.Conv2D(32, (3, 3), padding='same', name='block1_conv2')(x)
    x = layers.BatchNormalization(name='block1_bn2')(x)
    x = layers.ReLU(name='block1_relu2')(x)
    x = layers.MaxPooling2D((2, 2), name='block1_pool')(x)
    x = layers.Dropout(0.25, name='block1_dropout')(x)

    # Block 2
    x = layers.Conv2D(64, (3, 3), padding='same', name='block2_conv1')(x)
    x = layers.BatchNormalization(name='block2_bn1')(x)
    x = layers.ReLU(name='block2_relu1')(x)
    x = layers.Conv2D(64, (3, 3), padding='same', name='block2_conv2')(x)
    x = layers.BatchNormalization(name='block2_bn2')(x)
    x = layers.ReLU(name='block2_relu2')(x)
    x = layers.MaxPooling2D((2, 2), name='block2_pool')(x)
    x = layers.Dropout(0.25, name='block2_dropout')(x)

    # Block 3
    x = layers.Conv2D(128, (3, 3), padding='same', name='block3_conv1')(x)
    x = layers.BatchNormalization(name='block3_bn1')(x)
    x = layers.ReLU(name='block3_relu1')(x)
    x = layers.Conv2D(128, (3, 3), padding='same', name='block3_conv2')(x)
    x = layers.BatchNormalization(name='block3_bn2')(x)
    x = layers.ReLU(name='block3_relu2')(x)
    x = layers.MaxPooling2D((2, 2), name='block3_pool')(x)
    x = layers.Dropout(0.30, name='block3_dropout')(x)

    # Block 4
    x = layers.Conv2D(256, (3, 3), padding='same', name='block4_conv')(x)
    x = layers.BatchNormalization(name='block4_bn')(x)
    x = layers.ReLU(name='block4_relu')(x)
    x = layers.MaxPooling2D((2, 2), name='block4_pool')(x)
    x = layers.Dropout(0.35, name='block4_dropout')(x)

    # Head
    x = layers.GlobalAveragePooling2D(name='gap')(x)
    x = layers.Dense(128, name='fc1')(x)
    x = layers.ReLU(name='fc1_relu')(x)
    x = layers.Dropout(0.5, name='fc1_dropout')(x)
    outputs = layers.Dense(1, activation='sigmoid', name='output')(x)

    model = Model(inputs=inputs, outputs=outputs, name='ScratchCNN')

    return model


def create_mobilenet_model(input_shape: tuple = None, trainable: bool = False) -> tuple:
    """
    Create a MobileNetV2 transfer learning model.

    Args:
        input_shape: Input image shape
        trainable: Whether base model is trainable (False for freeze phase)

    Returns:
        Tuple of (model, base_model)
    """
    if input_shape is None:
        input_shape = config.IMG_SHAPE

    # Load pretrained MobileNetV2
    base_model = keras.applications.MobileNetV2(
        input_shape=input_shape,
        include_top=False,
        weights='imagenet'
    )
    base_model.trainable = trainable

    # Build model with custom head
    inputs = layers.Input(shape=input_shape, name='input')

    # Preprocessing for MobileNet
    x = keras.applications.mobilenet_v2.preprocess_input(inputs)
    x = base_model(x, training=trainable)

    # Custom head
    x = layers.GlobalAveragePooling2D(name='gap')(x)
    x = layers.Dropout(0.4, name='head_dropout')(x)
    outputs = layers.Dense(1, activation='sigmoid', name='output')(x)

    model = Model(inputs=inputs, outputs=outputs, name='MobileNetV2_TL')

    return model, base_model


def create_efficientnet_model(input_shape: tuple = None, trainable: bool = False) -> tuple:
    """
    Create an EfficientNetB0 transfer learning model.

    Args:
        input_shape: Input image shape
        trainable: Whether base model is trainable

    Returns:
        Tuple of (model, base_model)
    """
    if input_shape is None:
        input_shape = config.IMG_SHAPE

    # Load pretrained EfficientNetB0
    base_model = keras.applications.EfficientNetB0(
        input_shape=input_shape,
        include_top=False,
        weights='imagenet'
    )
    base_model.trainable = trainable

    # Build model with custom head
    inputs = layers.Input(shape=input_shape, name='input')

    # EfficientNet has built-in preprocessing
    x = base_model(inputs, training=trainable)

    # Custom head
    x = layers.GlobalAveragePooling2D(name='gap')(x)
    x = layers.Dropout(0.4, name='head_dropout')(x)
    outputs = layers.Dense(1, activation='sigmoid', name='output')(x)

    model = Model(inputs=inputs, outputs=outputs, name='EfficientNetB0_TL')

    return model, base_model


def unfreeze_top_layers(base_model: Model, ratio: float = None) -> int:
    """
    Unfreeze the top percentage of layers for fine-tuning.

    Args:
        base_model: The base model to unfreeze
        ratio: Ratio of layers to unfreeze (default: from config)

    Returns:
        Number of layers unfrozen
    """
    if ratio is None:
        ratio = config.FINETUNE_RATIO

    total_layers = len(base_model.layers)
    fine_tune_from = int((1 - ratio) * total_layers)

    # Keep BatchNorm layers frozen (recommended for fine-tuning)
    unfrozen_count = 0
    for i, layer in enumerate(base_model.layers):
        if i >= fine_tune_from:
            if not isinstance(layer, layers.BatchNormalization):
                layer.trainable = True
                unfrozen_count += 1
            # Keep BatchNorm frozen for stability
        else:
            layer.trainable = False

    print(f"\nFine-tuning configuration:")
    print(f"  Total layers: {total_layers}")
    print(f"  Frozen layers: {fine_tune_from}")
    print(f"  Unfrozen layers: {unfrozen_count}")
    print(f"  BatchNorm layers: kept frozen for stability")

    return unfrozen_count


def compile_model(
    model: Model,
    learning_rate: float,
    show_summary: bool = True
) -> Model:
    """
    Compile a model with BinaryCrossentropy and Adam optimizer.

    Args:
        model: Keras model to compile
        learning_rate: Learning rate for Adam optimizer
        show_summary: Whether to print model summary

    Returns:
        Compiled model
    """
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss=keras.losses.BinaryCrossentropy(),
        metrics=[
            keras.metrics.BinaryAccuracy(name='accuracy'),
            keras.metrics.AUC(name='auc'),
        ]
    )

    if show_summary:
        print(f"\n{'='*60}")
        print(f"Model: {model.name}")
        print(f"Learning Rate: {learning_rate}")
        print('='*60)
        model.summary()

    return model


def get_model_info(model: Model) -> dict:
    """
    Get model information for reporting.

    Args:
        model: Keras model

    Returns:
        Dictionary with model info
    """
    trainable_params = sum([tf.reduce_prod(w.shape).numpy() for w in model.trainable_weights])
    non_trainable_params = sum([tf.reduce_prod(w.shape).numpy() for w in model.non_trainable_weights])

    return {
        'name': model.name,
        'total_params': trainable_params + non_trainable_params,
        'trainable_params': trainable_params,
        'non_trainable_params': non_trainable_params,
        'layers': len(model.layers)
    }


if __name__ == "__main__":
    # Test model creation
    print("Testing Scratch CNN...")
    scratch_model = create_scratch_cnn()
    compile_model(scratch_model, config.SCRATCH_LR)

    print("\nTesting MobileNetV2...")
    mobilenet_model, mobilenet_base = create_mobilenet_model()
    compile_model(mobilenet_model, config.TL_FREEZE_LR)

    print("\nTesting EfficientNetB0...")
    efficientnet_model, efficientnet_base = create_efficientnet_model()
    compile_model(efficientnet_model, config.TL_FREEZE_LR)


Writing src/models.py


In [8]:
%%writefile src/training.py
"""
ISIC 2018 Skin Lesion Classification - Training Module
"""
import os
import json
from typing import Optional, Tuple, Dict
from datetime import datetime

import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

from . import config


def get_callbacks(
    model_name: str,
    monitor: str = 'val_loss',
    patience_early: int = None,
    patience_lr: int = None,
) -> list:
    """
    Create training callbacks.

    Args:
        model_name: Name for checkpoint file
        monitor: Metric to monitor
        patience_early: EarlyStopping patience
        patience_lr: ReduceLROnPlateau patience

    Returns:
        List of callbacks
    """
    if patience_early is None:
        patience_early = config.EARLY_STOPPING_PATIENCE
    if patience_lr is None:
        patience_lr = config.REDUCE_LR_PATIENCE

    callbacks = [
        keras.callbacks.EarlyStopping(
            monitor=monitor,
            patience=patience_early,
            restore_best_weights=True,
            verbose=1
        ),
        keras.callbacks.ReduceLROnPlateau(
            monitor=monitor,
            factor=config.REDUCE_LR_FACTOR,
            patience=patience_lr,
            min_lr=config.MIN_LR,
            verbose=1
        ),
        keras.callbacks.ModelCheckpoint(
            filepath=os.path.join(config.MODELS_DIR, f'{model_name}_best.keras'),
            monitor=monitor,
            save_best_only=True,
            verbose=1
        ),
    ]

    return callbacks


def train_model(
    model: keras.Model,
    train_ds: tf.data.Dataset,
    val_ds: tf.data.Dataset,
    epochs: int = None,
    callbacks: list = None,
    verbose: int = 1
) -> keras.callbacks.History:
    """
    Train a model.

    Args:
        model: Compiled Keras model
        train_ds: Training dataset
        val_ds: Validation dataset
        epochs: Number of epochs
        callbacks: List of callbacks
        verbose: Verbosity level

    Returns:
        Training history
    """
    if epochs is None:
        epochs = config.EPOCHS

    if callbacks is None:
        callbacks = get_callbacks(model.name)

    print(f"\n{'='*60}")
    print(f"Training: {model.name}")
    print(f"Epochs: {epochs}")
    print(f"{'='*60}\n")

    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=epochs,
        callbacks=callbacks,
        verbose=verbose
    )

    return history


def plot_training_history(
    history: keras.callbacks.History,
    model_name: str,
    save_path: str = None
):
    """
    Plot training and validation loss/accuracy curves.

    Args:
        history: Training history object
        model_name: Model name for title
        save_path: Path to save figure
    """
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    # Loss
    axes[0].plot(history.history['loss'], label='Train Loss', linewidth=2)
    axes[0].plot(history.history['val_loss'], label='Val Loss', linewidth=2)
    axes[0].set_title(f'{model_name} - Loss', fontsize=12)
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)

    # Accuracy
    axes[1].plot(history.history['accuracy'], label='Train Accuracy', linewidth=2)
    axes[1].plot(history.history['val_accuracy'], label='Val Accuracy', linewidth=2)
    axes[1].set_title(f'{model_name} - Accuracy', fontsize=12)
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Accuracy')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)

    plt.tight_layout()

    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        print(f"Training curves saved to: {save_path}")

    plt.close()


def save_history(history: keras.callbacks.History, model_name: str):
    """
    Save training history to JSON file.

    Args:
        history: Training history
        model_name: Model name
    """
    history_dict = {k: [float(v) for v in vals] for k, vals in history.history.items()}

    save_path = os.path.join(config.REPORTS_DIR, f'{model_name}_history.json')
    with open(save_path, 'w') as f:
        json.dump(history_dict, f, indent=2)

    print(f"Training history saved to: {save_path}")


def train_scratch_cnn(
    train_ds: tf.data.Dataset,
    val_ds: tf.data.Dataset,
    epochs: int = None
) -> Tuple[keras.Model, keras.callbacks.History]:
    """
    Train the Scratch CNN model.

    Args:
        train_ds: Training dataset
        val_ds: Validation dataset
        epochs: Number of epochs

    Returns:
        Tuple of (trained model, history)
    """
    from .models import create_scratch_cnn, compile_model

    model = create_scratch_cnn()
    model = compile_model(model, config.SCRATCH_LR)

    callbacks = get_callbacks('scratch_cnn')
    history = train_model(model, train_ds, val_ds, epochs, callbacks)

    # Save artifacts
    plot_training_history(
        history, 'Scratch CNN',
        os.path.join(config.FIGURES_DIR, 'scratch_cnn_training.png')
    )
    save_history(history, 'scratch_cnn')

    return model, history


def train_mobilenet(
    train_ds: tf.data.Dataset,
    val_ds: tf.data.Dataset,
    epochs_freeze: int = None,
    epochs_finetune: int = None
) -> Tuple[keras.Model, Dict]:
    """
    Train MobileNetV2 with freeze + fine-tune strategy.

    Args:
        train_ds: Training dataset
        val_ds: Validation dataset
        epochs_freeze: Epochs for freeze phase
        epochs_finetune: Epochs for fine-tune phase

    Returns:
        Tuple of (trained model, histories dict)
    """
    from .models import create_mobilenet_model, compile_model, unfreeze_top_layers

    if epochs_freeze is None:
        epochs_freeze = config.EPOCHS
    if epochs_finetune is None:
        epochs_finetune = config.EPOCHS

    histories = {}

    # Phase 1: Feature Extraction (Freeze)
    print("\n" + "="*60)
    print("MobileNetV2 - Phase 1: Feature Extraction (Freeze)")
    print("="*60)

    model, base_model = create_mobilenet_model(trainable=False)
    model = compile_model(model, config.TL_FREEZE_LR)

    callbacks = get_callbacks('mobilenet_freeze')
    history_freeze = train_model(model, train_ds, val_ds, epochs_freeze, callbacks)
    histories['freeze'] = history_freeze

    plot_training_history(
        history_freeze, 'MobileNetV2 - Freeze Phase',
        os.path.join(config.FIGURES_DIR, 'mobilenet_freeze_training.png')
    )
    save_history(history_freeze, 'mobilenet_freeze')

    # Phase 2: Fine-Tuning (Unfreeze top 25%)
    print("\n" + "="*60)
    print("MobileNetV2 - Phase 2: Fine-Tuning (Top 25%)")
    print("="*60)

    unfreeze_top_layers(base_model, config.FINETUNE_RATIO)
    model = compile_model(model, config.TL_FINETUNE_LR, show_summary=False)

    callbacks = get_callbacks('mobilenet_finetune')
    history_finetune = train_model(model, train_ds, val_ds, epochs_finetune, callbacks)
    histories['finetune'] = history_finetune

    plot_training_history(
        history_finetune, 'MobileNetV2 - Fine-Tune Phase',
        os.path.join(config.FIGURES_DIR, 'mobilenet_finetune_training.png')
    )
    save_history(history_finetune, 'mobilenet_finetune')

    # Save final model
    model.save(os.path.join(config.MODELS_DIR, 'mobilenet_final.keras'))

    return model, histories


def train_efficientnet(
    train_ds: tf.data.Dataset,
    val_ds: tf.data.Dataset,
    epochs_freeze: int = None,
    epochs_finetune: int = None
) -> Tuple[keras.Model, Dict]:
    """
    Train EfficientNetB0 with freeze + fine-tune strategy.

    Args:
        train_ds: Training dataset
        val_ds: Validation dataset
        epochs_freeze: Epochs for freeze phase
        epochs_finetune: Epochs for fine-tune phase

    Returns:
        Tuple of (trained model, histories dict)
    """
    from .models import create_efficientnet_model, compile_model, unfreeze_top_layers

    if epochs_freeze is None:
        epochs_freeze = config.EPOCHS
    if epochs_finetune is None:
        epochs_finetune = config.EPOCHS

    histories = {}

    # Phase 1: Feature Extraction (Freeze)
    print("\n" + "="*60)
    print("EfficientNetB0 - Phase 1: Feature Extraction (Freeze)")
    print("="*60)

    model, base_model = create_efficientnet_model(trainable=False)
    model = compile_model(model, config.TL_FREEZE_LR)

    callbacks = get_callbacks('efficientnet_freeze')
    history_freeze = train_model(model, train_ds, val_ds, epochs_freeze, callbacks)
    histories['freeze'] = history_freeze

    plot_training_history(
        history_freeze, 'EfficientNetB0 - Freeze Phase',
        os.path.join(config.FIGURES_DIR, 'efficientnet_freeze_training.png')
    )
    save_history(history_freeze, 'efficientnet_freeze')

    # Phase 2: Fine-Tuning (Unfreeze top 25%)
    print("\n" + "="*60)
    print("EfficientNetB0 - Phase 2: Fine-Tuning (Top 25%)")
    print("="*60)

    unfreeze_top_layers(base_model, config.FINETUNE_RATIO)
    model = compile_model(model, config.TL_FINETUNE_LR, show_summary=False)

    callbacks = get_callbacks('efficientnet_finetune')
    history_finetune = train_model(model, train_ds, val_ds, epochs_finetune, callbacks)
    histories['finetune'] = history_finetune

    plot_training_history(
        history_finetune, 'EfficientNetB0 - Fine-Tune Phase',
        os.path.join(config.FIGURES_DIR, 'efficientnet_finetune_training.png')
    )
    save_history(history_finetune, 'efficientnet_finetune')

    # Save final model
    model.save(os.path.join(config.MODELS_DIR, 'efficientnet_final.keras'))

    return model, histories


Writing src/training.py


In [9]:
%%writefile src/evaluation.py
"""
ISIC 2018 Skin Lesion Classification - Evaluation Module
"""
import os
import json
from typing import List, Dict, Tuple

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, roc_curve, confusion_matrix, classification_report
)

from . import config


def predict_on_dataset(
    model: keras.Model,
    dataset: tf.data.Dataset
) -> Tuple[np.ndarray, np.ndarray]:
    """
    Get predictions and true labels from a dataset.

    Args:
        model: Trained model
        dataset: tf.data.Dataset

    Returns:
        Tuple of (y_true, y_pred_proba)
    """
    y_true = []
    y_pred_proba = []

    for images, labels in dataset:
        preds = model.predict(images, verbose=0)
        y_true.extend(labels.numpy())
        y_pred_proba.extend(preds.flatten())

    return np.array(y_true), np.array(y_pred_proba)


def calculate_metrics(
    y_true: np.ndarray,
    y_pred_proba: np.ndarray,
    threshold: float = 0.5
) -> Dict[str, float]:
    """
    Calculate classification metrics.

    Args:
        y_true: True labels
        y_pred_proba: Predicted probabilities
        threshold: Classification threshold

    Returns:
        Dictionary of metrics
    """
    y_pred = (y_pred_proba >= threshold).astype(int)

    metrics = {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred, zero_division=0),
        'recall': recall_score(y_true, y_pred, zero_division=0),
        'f1_score': f1_score(y_true, y_pred, zero_division=0),
        'roc_auc': roc_auc_score(y_true, y_pred_proba),
    }

    return metrics


def plot_confusion_matrix(
    y_true: np.ndarray,
    y_pred_proba: np.ndarray,
    class_names: Tuple[str, str],
    model_name: str,
    save_path: str = None,
    threshold: float = 0.5
):
    """
    Plot confusion matrix.

    Args:
        y_true: True labels
        y_pred_proba: Predicted probabilities
        class_names: Tuple of class names
        model_name: Model name for title
        save_path: Path to save figure
        threshold: Classification threshold
    """
    y_pred = (y_pred_proba >= threshold).astype(int)
    cm = confusion_matrix(y_true, y_pred)

    plt.figure(figsize=(8, 6))
    sns.heatmap(
        cm, annot=True, fmt='d', cmap='Blues',
        xticklabels=class_names,
        yticklabels=class_names
    )
    plt.title(f'Confusion Matrix - {model_name}', fontsize=14)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.tight_layout()

    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        print(f"Confusion matrix saved to: {save_path}")

    plt.close()


def plot_roc_curve(
    results: Dict[str, Tuple[np.ndarray, np.ndarray]],
    save_path: str = None
):
    """
    Plot ROC curves for multiple models.

    Args:
        results: Dict of model_name -> (y_true, y_pred_proba)
        save_path: Path to save figure
    """
    plt.figure(figsize=(10, 8))

    colors = ['#1f77b4', '#ff7f0e', '#2ca02c']

    for i, (model_name, (y_true, y_pred_proba)) in enumerate(results.items()):
        fpr, tpr, _ = roc_curve(y_true, y_pred_proba)
        auc = roc_auc_score(y_true, y_pred_proba)
        plt.plot(fpr, tpr, color=colors[i], linewidth=2,
                label=f'{model_name} (AUC = {auc:.4f})')

    plt.plot([0, 1], [0, 1], 'k--', linewidth=1, label='Random')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate', fontsize=12)
    plt.ylabel('True Positive Rate', fontsize=12)
    plt.title('ROC Curves - Model Comparison', fontsize=14)
    plt.legend(loc='lower right', fontsize=10)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()

    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        print(f"ROC curves saved to: {save_path}")

    plt.close()


def create_comparison_table(
    metrics_dict: Dict[str, Dict[str, float]],
    save_path: str = None
) -> pd.DataFrame:
    """
    Create a comparison table of metrics.

    Args:
        metrics_dict: Dict of model_name -> metrics
        save_path: Path to save CSV

    Returns:
        DataFrame with comparison
    """
    df = pd.DataFrame(metrics_dict).T
    df.index.name = 'Model'
    df = df.round(4)

    # Reorder columns
    col_order = ['accuracy', 'precision', 'recall', 'f1_score', 'roc_auc']
    df = df[col_order]

    print("\n" + "="*80)
    print("MODEL COMPARISON - TEST SET METRICS")
    print("="*80)
    print(df.to_string())
    print("="*80)

    # Highlight best model
    best_f1 = df['f1_score'].idxmax()
    best_auc = df['roc_auc'].idxmax()
    print(f"\nBest F1-Score: {best_f1} ({df.loc[best_f1, 'f1_score']:.4f})")
    print(f"Best ROC-AUC: {best_auc} ({df.loc[best_auc, 'roc_auc']:.4f})")

    if save_path:
        df.to_csv(save_path)
        print(f"\nComparison table saved to: {save_path}")

    return df


def print_classification_report(
    y_true: np.ndarray,
    y_pred_proba: np.ndarray,
    class_names: Tuple[str, str],
    model_name: str,
    threshold: float = 0.5
):
    """
    Print detailed classification report.

    Args:
        y_true: True labels
        y_pred_proba: Predicted probabilities
        class_names: Tuple of class names
        model_name: Model name
        threshold: Classification threshold
    """
    y_pred = (y_pred_proba >= threshold).astype(int)

    print(f"\n{'='*60}")
    print(f"Classification Report - {model_name}")
    print("="*60)
    print(classification_report(y_true, y_pred, target_names=list(class_names)))


def evaluate_model(
    model: keras.Model,
    test_ds: tf.data.Dataset,
    class_names: Tuple[str, str],
    model_name: str
) -> Tuple[Dict[str, float], np.ndarray, np.ndarray]:
    """
    Complete evaluation of a model.

    Args:
        model: Trained model
        test_ds: Test dataset
        class_names: Tuple of class names
        model_name: Model name

    Returns:
        Tuple of (metrics, y_true, y_pred_proba)
    """
    print(f"\n{'='*60}")
    print(f"Evaluating: {model_name}")
    print("="*60)

    # Get predictions
    y_true, y_pred_proba = predict_on_dataset(model, test_ds)

    # Calculate metrics
    metrics = calculate_metrics(y_true, y_pred_proba)

    # Plot confusion matrix
    plot_confusion_matrix(
        y_true, y_pred_proba, class_names, model_name,
        save_path=os.path.join(config.FIGURES_DIR, f'{model_name.lower().replace(" ", "_")}_confusion_matrix.png')
    )

    # Print classification report
    print_classification_report(y_true, y_pred_proba, class_names, model_name)

    # Print metrics
    print(f"\nMetrics Summary:")
    for metric_name, value in metrics.items():
        print(f"  {metric_name}: {value:.4f}")

    return metrics, y_true, y_pred_proba


def evaluate_all_models(
    models: Dict[str, keras.Model],
    test_ds: tf.data.Dataset,
    class_names: Tuple[str, str]
) -> Dict[str, Dict[str, float]]:
    """
    Evaluate all models and create comparison.

    Args:
        models: Dict of model_name -> model
        test_ds: Test dataset
        class_names: Tuple of class names

    Returns:
        Dict of model_name -> metrics
    """
    all_metrics = {}
    all_predictions = {}

    for model_name, model in models.items():
        metrics, y_true, y_pred_proba = evaluate_model(
            model, test_ds, class_names, model_name
        )
        all_metrics[model_name] = metrics
        all_predictions[model_name] = (y_true, y_pred_proba)

    # Plot combined ROC curves
    plot_roc_curve(
        all_predictions,
        save_path=os.path.join(config.FIGURES_DIR, 'roc_curves_comparison.png')
    )

    # Create comparison table
    comparison_df = create_comparison_table(
        all_metrics,
        save_path=os.path.join(config.REPORTS_DIR, 'comparison_table.csv')
    )

    # Save all predictions for potential later use
    for model_name, (y_true, y_pred_proba) in all_predictions.items():
        save_path = os.path.join(config.REPORTS_DIR, f'{model_name.lower().replace(" ", "_")}_predictions.npz')
        np.savez(save_path, y_true=y_true, y_pred_proba=y_pred_proba)

    return all_metrics


Writing src/evaluation.py


In [10]:
%%writefile src/gradcam.py
"""
ISIC 2018 Skin Lesion Classification - Grad-CAM Module
"""
import os
from typing import List, Tuple

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

from . import config


def make_gradcam_heatmap(
    img_array: np.ndarray,
    model: keras.Model,
) -> np.ndarray:
    """
    Generate Grad-CAM heatmap for an image.
    Works with both regular CNN and transfer learning models.

    Args:
        img_array: Preprocessed image array (1, H, W, C)
        model: Trained model

    Returns:
        Heatmap array
    """
    # Find the layer before GlobalAveragePooling
    gap_idx = None
    for i, layer in enumerate(model.layers):
        if isinstance(layer, keras.layers.GlobalAveragePooling2D):
            gap_idx = i
            break

    if gap_idx is None or gap_idx == 0:
        # Fallback: find any layer with spatial dimensions
        for i, layer in enumerate(reversed(model.layers)):
            if hasattr(layer, 'output') and len(layer.output.shape) == 4:
                gap_idx = len(model.layers) - i
                break

    if gap_idx is None or gap_idx == 0:
        raise ValueError("Could not find suitable layer for Grad-CAM")

    # Get the layer just before GAP
    target_layer = model.layers[gap_idx - 1]

    # Build activation model
    activation_model = keras.Model(
        inputs=model.input,
        outputs=[target_layer.output, model.output]
    )

    # Get activations and predictions
    with tf.GradientTape() as tape:
        activations, predictions = activation_model(img_array)
        tape.watch(activations)
        loss = predictions[:, 0]

    # Get gradients
    grads = tape.gradient(loss, activations)

    if grads is None:
        # Fallback: use activation magnitude
        activations = activations[0]
        heatmap = tf.reduce_mean(activations, axis=-1)
    else:
        # Weighted activation map
        pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
        activations = activations[0]
        heatmap = activations @ pooled_grads[..., tf.newaxis]
        heatmap = tf.squeeze(heatmap)

    # Normalize
    heatmap = tf.maximum(heatmap, 0)
    max_val = tf.math.reduce_max(heatmap)
    if max_val > 0:
        heatmap = heatmap / max_val

    return heatmap.numpy()


def create_gradcam_visualization(
    image: np.ndarray,
    heatmap: np.ndarray,
    alpha: float = 0.4
) -> Tuple[np.ndarray, np.ndarray]:
    """
    Create overlay of Grad-CAM heatmap on original image.

    Args:
        image: Original image (H, W, C) in [0, 1]
        heatmap: Grad-CAM heatmap
        alpha: Overlay transparency

    Returns:
        Tuple of (overlay, resized_heatmap)
    """
    # Resize heatmap to image size
    heatmap_resized = tf.image.resize(
        heatmap[..., np.newaxis],
        (image.shape[0], image.shape[1])
    ).numpy().squeeze()

    # Convert heatmap to RGB using colormap
    heatmap_colored = plt.cm.jet(heatmap_resized)[:, :, :3]

    # Create overlay
    overlay = (1 - alpha) * image + alpha * heatmap_colored
    overlay = np.clip(overlay, 0, 1)

    return overlay, heatmap_resized


def load_and_preprocess_image(path: str) -> Tuple[np.ndarray, np.ndarray]:
    """
    Load and preprocess a single image for Grad-CAM.

    Args:
        path: Image file path

    Returns:
        Tuple of (original_image, preprocessed_image)
    """
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)

    # Keep original for visualization
    original = tf.image.resize(img, config.IMG_SIZE).numpy() / 255.0

    # Preprocess for model
    preprocessed = tf.cast(tf.image.resize(img, config.IMG_SIZE), tf.float32) / 255.0
    preprocessed = tf.expand_dims(preprocessed, 0)

    return original, preprocessed.numpy()


def select_samples_for_gradcam(
    model: keras.Model,
    test_paths: List[str],
    test_labels: List[int],
    num_correct: int = 3,
    num_incorrect: int = 3
) -> Tuple[List[dict], List[dict]]:
    """
    Select samples for Grad-CAM analysis.

    Args:
        model: Trained model
        test_paths: List of test image paths
        test_labels: List of test labels
        num_correct: Number of correctly classified samples
        num_incorrect: Number of incorrectly classified samples

    Returns:
        Tuple of (correct_samples, incorrect_samples)
    """
    correct_samples = []
    incorrect_samples = []

    for path, true_label in zip(test_paths, test_labels):
        if len(correct_samples) >= num_correct and len(incorrect_samples) >= num_incorrect:
            break

        # Load and predict
        _, preprocessed = load_and_preprocess_image(path)
        pred_proba = model.predict(preprocessed, verbose=0)[0, 0]
        pred_label = int(pred_proba >= 0.5)

        sample_info = {
            'path': path,
            'true_label': true_label,
            'pred_label': pred_label,
            'pred_proba': float(pred_proba),
        }

        if pred_label == true_label and len(correct_samples) < num_correct:
            correct_samples.append(sample_info)
        elif pred_label != true_label and len(incorrect_samples) < num_incorrect:
            incorrect_samples.append(sample_info)

    print(f"Selected {len(correct_samples)} correct and {len(incorrect_samples)} incorrect samples")

    return correct_samples, incorrect_samples


def generate_gradcam_for_samples(
    model: keras.Model,
    samples: List[dict],
    class_names: Tuple[str, str],
    model_name: str,
    sample_type: str,
    save_dir: str = None
):
    """
    Generate Grad-CAM visualizations for selected samples.

    Args:
        model: Trained model
        samples: List of sample info dicts
        class_names: Tuple of class names
        model_name: Model name
        sample_type: 'correct' or 'incorrect'
        save_dir: Directory to save figures
    """
    if save_dir is None:
        save_dir = config.GRADCAM_DIR

    print(f"Generating Grad-CAM for {model_name} ({sample_type})")

    for i, sample in enumerate(samples):
        original, preprocessed = load_and_preprocess_image(sample['path'])

        try:
            # Generate heatmap
            heatmap = make_gradcam_heatmap(preprocessed, model)

            # Create overlay
            overlay, heatmap_resized = create_gradcam_visualization(original, heatmap)

            # Plot
            fig, axes = plt.subplots(1, 3, figsize=(15, 5))

            # Original
            axes[0].imshow(original)
            axes[0].set_title('Original Image', fontsize=12)
            axes[0].axis('off')

            # Heatmap
            im = axes[1].imshow(heatmap_resized, cmap='jet')
            axes[1].set_title('Grad-CAM Heatmap', fontsize=12)
            axes[1].axis('off')
            plt.colorbar(im, ax=axes[1], fraction=0.046)

            # Overlay
            axes[2].imshow(overlay)
            axes[2].set_title('Overlay', fontsize=12)
            axes[2].axis('off')

            # Title
            true_class = class_names[sample['true_label']]
            pred_class = class_names[sample['pred_label']]
            confidence = sample['pred_proba'] if sample['pred_label'] == 1 else 1 - sample['pred_proba']

            fig.suptitle(
                f"{model_name} - {sample_type.capitalize()}\n"
                f"True: {true_class} | Pred: {pred_class} (Conf: {confidence:.2%})",
                fontsize=14
            )

            plt.tight_layout()

            # Save
            filename = f"{model_name.lower().replace(' ', '_')}_{sample_type}_{i+1}.png"
            save_path = os.path.join(save_dir, filename)
            plt.savefig(save_path, dpi=150, bbox_inches='tight')
            print(f"  Saved: {filename}")

            plt.close()

        except Exception as e:
            print(f"  Warning: Could not generate Grad-CAM for sample {i+1}: {e}")
            continue


def generate_all_gradcam(
    models: dict,
    test_paths: List[str],
    test_labels: List[int],
    class_names: Tuple[str, str]
):
    """
    Generate Grad-CAM visualizations for all models.

    Args:
        models: Dict of model_name -> model
        test_paths: List of test image paths
        test_labels: List of test labels
        class_names: Tuple of class names
    """
    print("\n" + "="*60)
    print("Generating Grad-CAM Visualizations")
    print("="*60)

    for model_name, model in models.items():
        print(f"\nProcessing: {model_name}")

        # Select samples
        correct_samples, incorrect_samples = select_samples_for_gradcam(
            model, test_paths, test_labels,
            num_correct=3, num_incorrect=3
        )

        # Generate visualizations
        if correct_samples:
            generate_gradcam_for_samples(
                model, correct_samples, class_names, model_name, 'correct'
            )

        if incorrect_samples:
            generate_gradcam_for_samples(
                model, incorrect_samples, class_names, model_name, 'incorrect'
            )

    print("\n" + "="*60)
    print("Grad-CAM generation complete!")
    print(f"Visualizations saved to: {config.GRADCAM_DIR}")
    print("="*60)


Writing src/gradcam.py


## 2. Pipeline Çalıştırma
Artık tüm modüller hazır. Pipeline'ı başlatabiliriz.

In [11]:
import sys
import os
import tensorflow as tf

# src klasörünü path'e ekle
sys.path.append('/content')

from src import config
from src.data_loader import prepare_data, visualize_augmentation
from src.training import train_scratch_cnn, train_mobilenet, train_efficientnet
from src.evaluation import evaluate_all_models
from src.gradcam import generate_all_gradcam

# kagglehub'dan indirilen veri seti yolunu kontrol et
print(f"Dataset path: {DATASET_PATH}")
print(f"Contents: {os.listdir(DATASET_PATH)}")

# Dizin yapısını otomatik tespit et
def find_data_dirs(base_path):
    '''Veri seti içindeki Train ve Test klasörlerini bul'''
    # Doğrudan base_path'te Train var mı?
    if os.path.isdir(os.path.join(base_path, 'Train')):
        return base_path

    # Alt klasörlerde ara
    for item in os.listdir(base_path):
        item_path = os.path.join(base_path, item)
        if os.path.isdir(item_path):
            if os.path.isdir(os.path.join(item_path, 'Train')):
                return item_path

    # Bulunamadıysa base_path döndür
    return base_path

data_root = find_data_dirs(DATASET_PATH)
print(f"Data root: {data_root}")
print(f"Data root contents: {os.listdir(data_root)}")

# config ayarlarını güncelle
config.DATA_DIR = data_root
config.TRAIN_DIR = os.path.join(data_root, 'Train')
config.TEST_DIR = os.path.join(data_root, 'Test')

print(f"\nTensorFlow Version: {tf.__version__}")
print(f"TRAIN_DIR: {config.TRAIN_DIR}")
print(f"TEST_DIR: {config.TEST_DIR}")
print(f"TRAIN_DIR exists: {os.path.exists(config.TRAIN_DIR)}")

Dataset path: /kaggle/input/skin-cancer9-classesisic
Contents: ['Skin cancer ISIC The International Skin Imaging Collaboration']
Data root: /kaggle/input/skin-cancer9-classesisic/Skin cancer ISIC The International Skin Imaging Collaboration
Data root contents: ['Test', 'Train']

TensorFlow Version: 2.19.0
TRAIN_DIR: /kaggle/input/skin-cancer9-classesisic/Skin cancer ISIC The International Skin Imaging Collaboration/Train
TEST_DIR: /kaggle/input/skin-cancer9-classesisic/Skin cancer ISIC The International Skin Imaging Collaboration/Test
TRAIN_DIR exists: True


In [12]:
# Veri Hazırlığı
train_ds, val_ds, test_ds, class_names, split_info = prepare_data()


ISIC 2018 Dataset - Class Distribution
                class_name  count
pigmented benign keratosis    462
                  melanoma    438
      basal cell carcinoma    376
                     nevus    357
   squamous cell carcinoma    181
           vascular lesion    139
         actinic keratosis    114
            dermatofibroma     95
      seborrheic keratosis     77

Total samples: 2239

Selected classes for binary classification:
  Class 0: pigmented benign keratosis
  Class 1: melanoma

Loaded 900 images:
  Class 0 (pigmented benign keratosis): 462
  Class 1 (melanoma): 438

Stratified Split Results
     TRAIN:  630 samples | Class 0: 324 | Class 1: 306
       VAL:  135 samples | Class 0:  69 | Class 1:  66
      TEST:  135 samples | Class 0:  69 | Class 1:  66


Augmentation examples saved to: /content/outputs/figures/augmentation_examples.png


In [13]:
# Augmentation Örnekleri
visualize_augmentation(train_ds, class_names)

In [14]:
# Model Eğitimi (Demo için epoch sayısını düşürebilirsiniz)
config.EPOCHS = 10  # Colab'da daha hızlı sonuç için
trained_models = {}

In [15]:
print("Training Scratch CNN...")
scratch_model, scratch_history = train_scratch_cnn(train_ds, val_ds)
trained_models['Scratch CNN'] = scratch_model

Training Scratch CNN...

Model: ScratchCNN
Learning Rate: 0.001


Model: "ScratchCNN"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input (InputLayer)              │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_bn1 (BatchNormalization) │ (None, 224, 224, 32)   │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_relu1 (ReLU)             │ (None, 224, 224, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_bn2 (BatchNormalization) │ (None, 224, 224, 32)   │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_relu2 (ReLU)             │ (None, 224, 224, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_dropout (Dropout)        │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_bn1 (BatchNormalization) │ (None, 112, 112, 64)   │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_relu1 (ReLU)             │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_bn2 (BatchNormalization) │ (None, 112, 112, 64)   │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_relu2 (ReLU)             │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_dropout (Dropout)        │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_bn1 (BatchNormalization) │ (None, 56, 56, 128)    │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_relu1 (ReLU)             │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_bn2 (BatchNormalization) │ (None, 56, 56, 128)    │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_relu2 (ReLU)             │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_dropout (Dropout)        │ (None, 28, 28, 128)    │             

 Total params: 618,017 (2.36 MB)

 Trainable params: 616,609 (2.35 MB)

 Non-trainable params: 1,408 (5.50 KB)


Training: ScratchCNN
Epochs: 10

Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 20s/step - accuracy: 0.5119 - auc: 0.5070 - loss: 0.8496 
Epoch 1: val_loss improved from inf to 0.69543, saving model to /content/outputs/models/scratch_cnn_best.keras
20/20 ━━━━━━━━━━━━━━━━━━━━ 434s 21s/step - accuracy: 0.5117 - auc: 0.5070 - loss: 0.8482 - val_accuracy: 0.3778 - val_auc: 0.3027 - val_loss: 0.6954 - learning_rate: 0.0010
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 19s/step - accuracy: 0.5039 - auc: 0.5158 - loss: 0.7391 
Epoch 2: val_loss did not improve from 0.69543
20/20 ━━━━━━━━━━━━━━━━━━━━ 402s 20s/step - accuracy: 0.5035 - auc: 0.5150 - loss: 0.7384 - val_accuracy: 0.4889 - val_auc: 0.7048 - val_loss: 0.6965 - learning_rate: 0.0010
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 20s/step - accuracy: 0.5367 - auc: 0.5375 - loss: 0.6954 
Epoch 3: val_loss improved from 0.69543 to 0.68949, saving model to /content/outputs/models/scratch_cnn_best.keras
20/20 ━━━━━━━━━━━━━━━━━━━━ 448s 20s/step - accurac

In [16]:
print("Training MobileNetV2...")
mobilenet_model, mobilenet_histories = train_mobilenet(train_ds, val_ds)
trained_models['MobileNetV2'] = mobilenet_model

Training MobileNetV2...

MobileNetV2 - Phase 1: Feature Extraction (Freeze)
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step

Model: MobileNetV2_TL
Learning Rate: 0.001


Model: "MobileNetV2_TL"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input (InputLayer)              │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ true_divide (TrueDivide)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ subtract (Subtract)             │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gap (GlobalAveragePooling2D)    │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ head_dropout (Dropout)          │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 1)              │         1,281 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,259,265 (8.62 MB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)


Training: MobileNetV2_TL
Epochs: 10

Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5102 - auc: 0.4992 - loss: 0.7565
Epoch 1: val_loss improved from inf to 0.69959, saving model to /content/outputs/models/mobilenet_freeze_best.keras
20/20 ━━━━━━━━━━━━━━━━━━━━ 60s 3s/step - accuracy: 0.5096 - auc: 0.4992 - loss: 0.7564 - val_accuracy: 0.4889 - val_auc: 0.5520 - val_loss: 0.6996 - learning_rate: 0.0010
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.4699 - auc: 0.4554 - loss: 0.7860
Epoch 2: val_loss improved from 0.69959 to 0.69911, saving model to /content/outputs/models/mobilenet_freeze_best.keras
20/20 ━━━━━━━━━━━━━━━━━━━━ 50s 2s/step - accuracy: 0.4713 - auc: 0.4572 - loss: 0.7846 - val_accuracy: 0.5111 - val_auc: 0.5747 - val_loss: 0.6991 - learning_rate: 0.0010
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.4906 - auc: 0.4628 - loss: 0.7713
Epoch 3: val_loss improved from 0.69911 to 0.69357, saving model to /content/outputs/models/m

In [17]:
print("Training EfficientNetB0...")
efficientnet_model, efficientnet_histories = train_efficientnet(train_ds, val_ds)
trained_models['EfficientNetB0'] = efficientnet_model

Training EfficientNetB0...

EfficientNetB0 - Phase 1: Feature Extraction (Freeze)
16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step

Model: EfficientNetB0_TL
Learning Rate: 0.001


Model: "EfficientNetB0_TL"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input (InputLayer)              │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gap (GlobalAveragePooling2D)    │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ head_dropout (Dropout)          │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 1)              │         1,281 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,050,852 (15.45 MB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 4,049,571 (15.45 MB)


Training: EfficientNetB0_TL
Epochs: 10

Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.5130 - auc: 0.5078 - loss: 0.7001
Epoch 1: val_loss improved from inf to 0.70199, saving model to /content/outputs/models/efficientnet_freeze_best.keras
20/20 ━━━━━━━━━━━━━━━━━━━━ 103s 4s/step - accuracy: 0.5121 - auc: 0.5064 - loss: 0.7005 - val_accuracy: 0.5111 - val_auc: 0.4621 - val_loss: 0.7020 - learning_rate: 0.0010
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.5204 - auc: 0.5013 - loss: 0.6997
Epoch 2: val_loss improved from 0.70199 to 0.69311, saving model to /content/outputs/models/efficientnet_freeze_best.keras
20/20 ━━━━━━━━━━━━━━━━━━━━ 138s 4s/step - accuracy: 0.5196 - auc: 0.5007 - loss: 0.7000 - val_accuracy: 0.5111 - val_auc: 0.5000 - val_loss: 0.6931 - learning_rate: 0.0010
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.5442 - auc: 0.4702 - loss: 0.7034
Epoch 3: val_loss did not improve from 0.69311
20/20 ━━━━━━━━━━━━━━━━━━━━ 143s 4s/

In [18]:
# Değerlendirme
results = evaluate_all_models(trained_models, test_ds, class_names)


Evaluating: Scratch CNN
Confusion matrix saved to: /content/outputs/figures/scratch_cnn_confusion_matrix.png

Classification Report - Scratch CNN
                            precision    recall  f1-score   support

pigmented benign keratosis       0.53      1.00      0.69        69
                  melanoma       1.00      0.06      0.11        66

                  accuracy                           0.54       135
                 macro avg       0.76      0.53      0.40       135
              weighted avg       0.76      0.54      0.41       135


Metrics Summary:
  accuracy: 0.5407
  precision: 1.0000
  recall: 0.0606
  f1_score: 0.1143
  roc_auc: 0.7578

Evaluating: MobileNetV2
Confusion matrix saved to: /content/outputs/figures/mobilenetv2_confusion_matrix.png

Classification Report - MobileNetV2
                            precision    recall  f1-score   support

pigmented benign keratosis       0.51      1.00      0.68        69
                  melanoma       0.00      0.00

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Confusion matrix saved to: /content/outputs/figures/efficientnetb0_confusion_matrix.png

Classification Report - EfficientNetB0
                            precision    recall  f1-score   support

pigmented benign keratosis       0.51      1.00      0.68        69
                  melanoma       0.00      0.00      0.00        66

                  accuracy                           0.51       135
                 macro avg       0.26      0.50      0.34       135
              weighted avg       0.26      0.51      0.35       135


Metrics Summary:
  accuracy: 0.5111
  precision: 0.0000
  recall: 0.0000
  f1_score: 0.0000
  roc_auc: 0.3722


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


ROC curves saved to: /content/outputs/figures/roc_curves_comparison.png

MODEL COMPARISON - TEST SET METRICS
                accuracy  precision  recall  f1_score  roc_auc
Model                                                         
Scratch CNN       0.5407        1.0  0.0606    0.1143   0.7578
MobileNetV2       0.5111        0.0  0.0000    0.0000   0.5470
EfficientNetB0    0.5111        0.0  0.0000    0.0000   0.3722

Best F1-Score: Scratch CNN (0.1143)
Best ROC-AUC: Scratch CNN (0.7578)

Comparison table saved to: /content/outputs/reports/comparison_table.csv


In [19]:
# Grad-CAM Analizi
generate_all_gradcam(
    trained_models,
    split_info['test_paths'],
    split_info['test_labels'],
    class_names
)


Generating Grad-CAM Visualizations

Processing: Scratch CNN
Selected 3 correct and 3 incorrect samples
Generating Grad-CAM for Scratch CNN (correct)
  Saved: scratch_cnn_correct_1.png
  Saved: scratch_cnn_correct_2.png
  Saved: scratch_cnn_correct_3.png
Generating Grad-CAM for Scratch CNN (incorrect)
  Saved: scratch_cnn_incorrect_1.png
  Saved: scratch_cnn_incorrect_2.png
  Saved: scratch_cnn_incorrect_3.png

Processing: MobileNetV2
Selected 3 correct and 3 incorrect samples
Generating Grad-CAM for MobileNetV2 (correct)
Generating Grad-CAM for MobileNetV2 (incorrect)

Processing: EfficientNetB0
Selected 3 correct and 3 incorrect samples
Generating Grad-CAM for EfficientNetB0 (correct)
Generating Grad-CAM for EfficientNetB0 (incorrect)

Grad-CAM generation complete!
Visualizations saved to: /content/outputs/gradcam


## Sonuçlar
Tüm görseller ve raporlar sol paneldeki `outputs` klasöründe bulunabilir. Bu klasörü sıkıştırıp indirebilirsiniz.

In [20]:
!zip -r outputs.zip outputs

  adding: outputs/ (stored 0%)
  adding: outputs/gradcam/ (stored 0%)
  adding: outputs/gradcam/scratch_cnn_incorrect_2.png (deflated 1%)
  adding: outputs/gradcam/scratch_cnn_correct_2.png (deflated 1%)
  adding: outputs/gradcam/scratch_cnn_correct_3.png (deflated 1%)
  adding: outputs/gradcam/scratch_cnn_incorrect_1.png (deflated 0%)
  adding: outputs/gradcam/scratch_cnn_correct_1.png (deflated 1%)
  adding: outputs/gradcam/scratch_cnn_incorrect_3.png (deflated 1%)
  adding: outputs/reports/ (stored 0%)
  adding: outputs/reports/mobilenet_finetune_history.json (deflated 68%)
  adding: outputs/reports/mobilenet_freeze_history.json (deflated 68%)
  adding: outputs/reports/efficientnetb0_predictions.npz (deflated 57%)
  adding: outputs/reports/comparison_table.csv (deflated 25%)
  adding: outputs/reports/mobilenetv2_predictions.npz (deflated 57%)
  adding: outputs/reports/efficientnet_finetune_history.json (deflated 69%)
  adding: outputs/reports/scratch_cnn_history.json (deflated 68%)
